# Using unlabelled, unfractionated datasets obtained from QExact and VOrbi instruments
* Datasets were searched against H_sapiens_Uniprot_SPROT_2017-04-12, Tryp_Pig_Bov sequence files using MSGFPlus
* Combined results with MASIC results (q <= 0.01) to get quantitation data

In [ ]:
import Classification_Utils as cu
import math
import numpy as np
from os import listdir
import pandas as pd
from sklearn.model_selection import train_test_split

## Load and combine data from all tissues

In [ ]:
df = pd.read_csv('FullPeptideQuant.txt', sep='\t', index_col='Peptide')
print(df.shape)

## Map each column to a corresponding label

In [ ]:
tissues = ['Blood_Plasma', 'Blood_Serum', 'CSF', 'Liver', 'Monocyte', 'Ovary', 'Pancreas', 'Substantia_Nigra', 'Temporal_Lobe']
 
tissues_to_columns = cu.map_tissues_to_columns(df, tissues)

In [ ]:
column_names = df.columns.values.tolist()
labels = cu.get_labels(column_names, tissues_to_columns)

## Make train-test split

In [ ]:
train_df, test_df, train_labels, test_labels = train_test_split(
    df.T, labels, test_size=0.30,# 30% of the data held out in test set
    random_state=0,    # Setting random_state ensures the same train/test split occurs each time this is run
    stratify=labels)   # Maintain relative ratio of samples from each tissue

original_train_df = train_df.copy()

## Transform Train Data by Reducing Peptides

In [ ]:
def keep_percentile_peptides(df, labels, tissues, percentile, impute_val):
    df = df.T
    tissue_dfs = []
    
    for tissue in tissues:
        cols_to_drop = [col for col in df.columns.values if not col.startswith(tissue)]
        tissue_df = df.drop(cols_to_drop, axis=1)
        
        ### Drop rows with nothing observed
        tissue_df.replace(impute_val, np.nan, inplace=True)
        tissue_df.dropna(axis=0, how='all', inplace=True)
        tissue_df.replace(np.nan, impute_val, inplace=True)
        
        peptide_mean_abundances = tissue_df.mean(axis=1) # pandas series

        # sort by average abundance value, drop below threshold
        peptide_mean_abundances.sort_values(ascending=False, inplace=True)
        num_peptides_to_keep = math.ceil(len(peptide_mean_abundances) * percentile/100)
        peptides_to_drop = peptide_mean_abundances[num_peptides_to_keep:].index.values
        
        print(tissue_df.shape[0], len(peptides_to_drop))
        
        tissue_df.drop(peptides_to_drop, axis=0, inplace=True)
        
        tissue_dfs.append(tissue_df)
        
    # join dataframes
    combined_df = pd.DataFrame()
    for next_df in tissue_dfs:
        combined_df = combined_df.join(next_df, how='outer')
    
    combined_df.replace(np.nan, impute_val, inplace=True)
        
    return combined_df.T

In [ ]:
imputed_val = train_df.mode().iloc[0, 0]

percentile_to_keep = 0.0007

print(train_df.shape)
train_df = keep_percentile_peptides(train_df, train_labels, tissues, percentile_to_keep, imputed_val)
print(train_df.shape)

features_to_keep = train_df.columns.values.tolist()

column_names = train_df.index.values.tolist()
train_labels = cu.get_labels(column_names, tissues_to_columns)

## Train classifiers on the reduced data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC

lr = LogisticRegression(random_state=0)
mnb = MultinomialNB()
rf = RandomForestClassifier(random_state=0)
svc = SVC(kernel='linear', random_state=0)
gbc = GradientBoostingClassifier(random_state=0)
gnb = GaussianNB()
knn = KNeighborsClassifier()

models = [lr, mnb, rf, svc, gbc, gnb, knn]

model_names = ['Logistic Regression', 'Multinomial Naive Bayes', 'Random Forest', 'SVC', 
               'Gradient Boosting', 'Gaussian Naive Bayes', 'K-Nearest Neighbors']

for model in models:
    model.fit(train_df, train_labels)

## Classify Training Set

In [ ]:
# Test data features (peptides) must match training data features exactly
# So we only keep the peptides still present in the training data after peptide reduction

print(test_df.shape)
test_df = test_df[features_to_keep]
print(test_df.shape)

### Use models from notebook to predict new data

In [ ]:
lr_pred = lr.predict(test_df)
lr_result = lr.score(test_df, test_labels)

mnb_pred = mnb.predict(test_df)
mnb_result = mnb.score(test_df, test_labels)

rf_pred = rf.predict(test_df)
rf_result = rf.score(test_df, test_labels)

svc_pred = svc.predict(test_df)
svc_result = svc.score(test_df, test_labels)

knn_pred = knn.predict(test_df)
knn_result = knn.score(test_df, test_labels)

gnb_pred = gnb.predict(test_df)
gnb_result = gnb.score(test_df, test_labels)

gbc_pred = gbc.predict(test_df)
gbc_result = gbc.score(test_df, test_labels)

In [ ]:
print(svc_result)
print(knn_result)
print(gnb_result)
print(gbc_result)
print(mnb_result)
print(lr_result)
print(rf_result)

In [ ]:
cm_labels = list(set(knn_pred.tolist() + test_labels))

cu.show_confusion_matrices(test_labels, knn_pred, cm_labels)